In [134]:
# Import libraries
import pandas as pd
import numpy as np
import ast
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import pickle

[nltk_data] Downloading package punkt to C:\Users\Syed
[nltk_data]     Raheel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
print(movies.shape)
movies.sample(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3583,10000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.ouridiotbrother.com/,59968,"[{""id"": 1155, ""name"": ""brother sister relation...",en,Our Idiot Brother,Everybody has the sibling who is always just a...,15.688439,"[{""name"": ""The Weinstein Company"", ""id"": 308},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-08-26,17273593,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Everybody has one,Our Idiot Brother,5.9,365


In [4]:
credits = pd.read_csv('tmdb_5000_credits.csv')

In [5]:
print(credits.shape)
credits.sample(1)

(4803, 4)


,movie_id,title,cast,crew
3808,114635,The Helpers,"[{""cast_id"": 1, ""character"": ""Claire"", ""credit...","[{""credit_id"": ""52fe4b53c3a36847f81fe1a9"", ""de..."


In [6]:
# join both data frames
df = pd.merge(movies,credits,left_on='id',right_on='movie_id')
print(df.shape)
df.sample(2)

(4803, 24)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
4696,8000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",http://weekenderfilm.tumblr.com/,79120,"[{""id"": 237, ""name"": ""gay""}, {""id"": 1025, ""nam...",en,Weekend,After a drunken house party with his straight ...,1.041254,"[{""name"": ""EM Media"", ""id"": 1917}, {""name"": ""T...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A (sort of) love story between two guys over a...,Weekend,7.4,163,79120,Weekend,"[{""cast_id"": 1000, ""character"": ""Russell"", ""cr...","[{""credit_id"": ""52fe49c2c3a368484e13e4e1"", ""de..."
2900,12000000,"[{""id"": 878, ""name"": ""Science Fiction""}]",NaN,61984,[],ja,Space Battleship Yamato,"In 2199, five years after the Gamilons began a...",5.158067,"[{""name"": ""Tokyo Broadcasting System (TBS)"", ""...",...,"[{""iso_639_1"": ""ja"", ""name"": ""\u65e5\u672c\u8a...",Released,NaN,Space Battleship Yamato,6.3,59,61984,Space Battleship Yamato,"[{""cast_id"": 1000, ""character"": ""Hajime Saito""...","[{""credit_id"": ""52fe466cc3a368484e08ffdf"", ""de..."


# Now Remove all useless features/columns from dataframe

In [7]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew'],
      dtype='object')

In [8]:
# Listing what we need to build recommender system
# 1) title , 2) id , 3) cast , 4) crew , 5) overview , 6) genres , 7) vote_average

df_filter = df[['title_x','id','cast','crew','overview','genres','vote_average']]

In [9]:
df_filter

,title_x,id,cast,crew,overview,genres,vote_average
0,Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2
1,Pirates of the Caribbean: At World's End,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9
2,Spectre,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3
3,The Dark Knight Rises,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6
4,John Carter,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1
...,...,...,...,...,...,...,...
4798,El Mariachi,9367,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",6.6
4799,Newlyweds,72766,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5.9
4800,"Signed, Sealed, Delivered",231617,"[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",7.0
4801,Shanghai Calling,126186,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",When ambitious New York attorney Sam is sent t...,[],5.7


## 2) Rearrange the columns

In [10]:
columns = {'id':'id','title_x':'title','genres':'genres','cast':'cast','crew':'crew','overview':'overview','vote_average':'vote'}
df_filter = df_filter.rename(columns=columns)
columns = ['id','title','genres','cast','crew','overview','vote']
df_filter = df_filter[columns]

In [11]:
df_filter.head(2)

,id,title,genres,cast,crew,overview,vote
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...",7.2
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...",6.9


In [12]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        4803 non-null   int64  
 1   title     4803 non-null   object 
 2   genres    4803 non-null   object 
 3   cast      4803 non-null   object 
 4   crew      4803 non-null   object 
 5   overview  4800 non-null   object 
 6   vote      4803 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 300.2+ KB


## Drop records containing null values

In [13]:
df_filter.dropna(inplace=True)
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 0 to 4802
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        4800 non-null   int64  
 1   title     4800 non-null   object 
 2   genres    4800 non-null   object 
 3   cast      4800 non-null   object 
 4   crew      4800 non-null   object 
 5   overview  4800 non-null   object 
 6   vote      4800 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 300.0+ KB


# Create list of genres

In [14]:
def genres(x):
    l = ast.literal_eval(x)
    y = []
    for i in range(len(l)):
        y.append(l[i]['name'])
    return y


In [15]:
df_filter['genres'] = df_filter['genres'].apply(genres)

In [16]:
df_filter.head(2)

,id,title,genres,cast,crew,overview,vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...",7.2
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...",6.9


# Create list of top 3 actors

In [17]:
def actor(x):
    l = ast.literal_eval(x)
    y = []
    counter = 1
    for i in range(len(l)):
        if counter == 4:
            break
        y.append(l[i]['name'])
        counter += 1
    return y

In [18]:
df_filter['actors'] = df_filter['cast'].apply(actor)

In [19]:
df_filter.drop(columns=['cast'],inplace=True)

In [20]:
df_filter.head(3)

,id,title,genres,crew,overview,vote,actors
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...",7.2,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...",6.9,"[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,206647,Spectre,"[Action, Adventure, Crime]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...,6.3,"[Daniel Craig, Christoph Waltz, Léa Seydoux]"


# Extract name of director from crew

In [21]:
def director(x):
    l = ast.literal_eval(x)
    y = ''
    for i in range(len(l)):
        if l[i]['job'] == 'Director':
            y = l[i]['name']
            break
    return y

In [22]:
df_filter['director'] = df_filter['crew'].apply(director)

In [23]:
df_filter.head(2)

,id,title,genres,crew,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...",7.2,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...",6.9,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski


In [24]:
df_filter.drop(columns=['crew'],axis=1,inplace=True)

In [25]:
df_filter.head(3)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...",7.2,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...",6.9,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,6.3,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes


In [26]:
df_filter['overview'] = df_filter['overview'].str.lower()
df_filter.head(2)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","in the 22nd century, a paraplegic marine is di...",7.2,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","captain barbossa, long believed to be dead, ha...",6.9,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski


In [27]:
# remove space between actor names
def space(x):
    y = []
    for i in range(len(x)):
        y.append(x[i].replace(' ',''))
    return y
    
df_filter.actors = df_filter['actors'].apply(space)
df_filter.head(2)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","in the 22nd century, a paraplegic marine is di...",7.2,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","captain barbossa, long believed to be dead, ha...",6.9,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",Gore Verbinski


In [28]:
df_filter.director = df_filter.director.apply(lambda x:x.replace(' ',''))

In [29]:
df_filter.head(2)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","in the 22nd century, a paraplegic marine is di...",7.2,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","captain barbossa, long believed to be dead, ha...",6.9,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",GoreVerbinski


In [30]:
df = df_filter.copy()

In [31]:
dic_dir = {}
y = 1
for i in range(df.shape[0]):
    x = df['director'].iloc[i]
    if x not in dic_dir:
        dic_dir.update({x:y})
        y+=1
        
df.director = df.director.apply(lambda x: dic_dir[x])

x_min = 1
x_max = 2347
x_diff = x_max-x_min
df.director = df.director.apply(lambda x:((x-x_min)/x_diff))
df.head(2)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","in the 22nd century, a paraplegic marine is di...",7.2,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",0.000000
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","captain barbossa, long believed to be dead, ha...",6.9,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",0.000426


In [32]:
dic_actors = {}
y = 1
for i in range(df.shape[0]):
    l = df['actors'].iloc[i]
    for j in range(len(l)):
        x = l[j]
        if x not in dic_actors:
            dic_actors.update({x:y})
            y+=1

def map_actors(x):
    y = []
    for i in range(len(x)):
        y.append(dic_actors[x[i]])
    return y

df.actors = df.actors.apply(map_actors)

x_min = min(dic_actors.values())
x_max = max(dic_actors.values())
x_diff = x_max - x_min

def normalize_actors(x):
    y = []
    for i in range(len(x)):
        y_norm = ((x[i]-x_min)/x_diff)
        y.append(y_norm)
    return y

df.actors = df.actors.apply(normalize_actors)

df.head(2)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","in the 22nd century, a paraplegic marine is di...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","captain barbossa, long believed to be dead, ha...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426


In [33]:
df.head(5)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","in the 22nd century, a paraplegic marine is di...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","captain barbossa, long believed to be dead, ha...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426
2,206647,Spectre,"[Action, Adventure, Crime]",a cryptic message from bond’s past sends him o...,6.3,"[0.0010559662090813093, 0.0012319605772615277,...",0.000853
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",following the death of district attorney harve...,7.6,"[0.0015839493136219642, 0.0017599436818021823,...",0.001279
4,49529,John Carter,"[Action, Adventure, Science Fiction]","john carter is a war-weary, former military ca...",6.1,"[0.0021119324181626186, 0.0022879267863428372,...",0.001705


In [34]:
dic_genres = {}
y = 1
for i in range(df.shape[0]):
    l = df['genres'].iloc[i]
    for j in range(len(l)):
        x = l[j]
        if x not in dic_genres:
            dic_genres.update({x:y})
            y+=1

def map_genres(x):
    y = []
    for i in range(len(x)):
        y.append(dic_genres[x[i]])
    return y

df.genres = df.genres.apply(map_genres)

x_min = min(dic_genres.values())
x_max = max(dic_genres.values())
x_diff = x_max - x_min

def normalize_genres(x):
    y = []
    for i in range(len(x)):
        y_norm = ((x[i]-x_min)/x_diff)
        y.append(y_norm)
    return y

df.genres = df.genres.apply(normalize_genres)

df.head(2)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[0.0, 0.05263157894736842, 0.10526315789473684...","in the 22nd century, a paraplegic marine is di...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000
1,285,Pirates of the Caribbean: At World's End,"[0.05263157894736842, 0.10526315789473684, 0.0]","captain barbossa, long believed to be dead, ha...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426


In [35]:
# Tokenize the descriptions into individual words:
df_filter['overview'] = df_filter['overview'].apply(nltk.word_tokenize)
# Remove stopwords:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df_filter['overview'] = df_filter['overview'].apply(lambda x: [word for word in x if word not in stop_words])
# Apply lemmatization:
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
df_filter['overview'] = df_filter['overview'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df_filter.head(2)

[nltk_data] Downloading package stopwords to C:\Users\Syed
[nltk_data]     Raheel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Syed
[nltk_data]     Raheel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Syed
[nltk_data]     Raheel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[22nd, century, ,, paraplegic, marine, dispatc...",7.2,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[captain, barbossa, ,, long, believed, dead, ,...",6.9,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",GoreVerbinski


In [36]:
dic_overview = {}
y = 1
for i in range(df.shape[0]):
    l = df['overview'].iloc[i].split(' ')
    for j in range(len(l)):
        x = l[j]
        if x not in dic_overview:
            dic_overview.update({x:y})
            y+=1

            
def map_overview(x):
    y = []
    x = x.split(' ')
    for i in range(len(x)):
        y.append(dic_overview[x[i]])
    return y

df.overview = df.overview.apply(map_overview)


x_min = min(dic_overview.values())
x_max = max(dic_overview.values())
x_diff = x_max - x_min


def normalize_overview(x):
    y = []
    for i in range(len(x)):
        y_norm = ((x[i]-x_min)/x_diff)
        y.append(y_norm)
    return y

df['overview'] = df.overview.apply(normalize_overview)

df.head(2)

,id,title,genres,overview,vote,actors,director
0,19995,Avatar,"[0.0, 0.05263157894736842, 0.10526315789473684...","[0.0, 2.9169826731229216e-05, 5.83396534624584...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000
1,285,Pirates of the Caribbean: At World's End,"[0.05263157894736842, 0.10526315789473684, 0.0]","[0.0007584154950119596, 0.0007875853217431889,...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426


In [37]:
l_gen = []
l_act = []
l_ove = []
for i in range(df.shape[0]):
    l_gen.append(len(df.genres.iloc[i]))
    l_act.append(len(df.actors.iloc[i]))
    l_ove.append(len(df.overview.iloc[i]))
    
print('-----------------------------')
print('max genres',max(l_gen))
print('max actors',max(l_act))
print('max overview',max(l_ove))
print('-----------------------------')
print('min genres',min(l_gen))
print('min actors',min(l_act))
print('min overview',min(l_ove))

-----------------------------
max genres 7
max actors 3
max overview 175
-----------------------------
min genres 0
min actors 0
min overview 2


In [38]:
emp_gen = []
emp_act = []
emp_ove = []
for i in range(df.shape[0]):
    if len(df.genres.iloc[i]) == 0:
        emp_gen.append(i)
    if len(df.actors.iloc[i]) == 0:
        emp_act.append(i)
    if len(df.overview.iloc[i]) == 0:
        emp_ove.append(i)

print('----------------------------------')
print('no of records where genres is null = ',len(emp_gen))
print('no of records where actors is null = ',len(emp_act))
print('no of records where overview is null = ',len(emp_ove))

# z = df.shape[0]
# for i in range(0,z):
    
#     if len(df.genres.iloc[i]) == 0:
#         df.drop(df.index[i],inplace=True)
#         z-=1
#     elif len(df.actors.iloc[i]) == 0:
#         df.drop(df.index[i],inplace=True)
#         z-=1
#     elif len(df.overview.iloc[i]) == 0:
#         df.drop(df.index[i],inplace=True)
#         z-=1
        
        
# emp_gen = []
# emp_act = []
# emp_ove = []
# for i in range(df.shape[0]):
#     if len(df.genres.iloc[i]) == 0:
#         emp_gen.append(i)
#     if len(df.actors.iloc[i]) == 0:
#         emp_act.append(i)
#     if len(df.overview.iloc[i]) == 0:
#         emp_ove.append(i)

# print('----------------------------------')
# print('no of records where genres is null = ',len(emp_gen))
# print('no of records where actors is null = ',len(emp_act))
# print('no of records where overview is null = ',len(emp_ove))

----------------------------------
no of records where genres is null =  28
no of records where actors is null =  42
no of records where overview is null =  0


In [39]:
g = []
for i in range(len(emp_gen)):
  g.append(df.genres.iloc[emp_gen[i]])

a = []
for i in range(len(emp_act)):
  a.append(df.actors.iloc[emp_act[i]])

o = []
for i in range(len(emp_ove)):
  o.append(df.overview.iloc[emp_ove[i]])

In [ ]:
def pad_genres(x):
  gen = x
  l = len(x)
  diff_gen = 7-l
  if diff_gen != 0:
    for i in range(0,diff_gen):
      gen.append(0)
  return gen

def pad_actors(x):
  act = x
  l = len(x)
  diff_act = 3-l
  if diff_act != 0:
    for i in range(0,diff_act):
      act.append(0)
  return act

def pad_overview(x):
  ove = x
  l = len(x)
  diff_ove = 175-l
  if diff_ove != 0:
    for i in range(0,diff_ove):
      ove.append(0)
  return ove

In [ ]:
df['similarity_genres'] = [[]] * len(df)
df['similarity_actors'] = [[]] * len(df)
df['similarity_overview'] = [[]] * len(df)

In [ ]:
for i in range(0,df.shape[0]):
    a = np.array(df.genres.iloc[i]).reshape(1,-1)
    l = []
    for j in range(0,df.shape[0]):
        b = np.array(df.genres.iloc[j]).reshape(1,-1)
        y = cosine_similarity(a,b)
        z = y[0][0]
        l.append(z)
    df['similarity_genres'].iloc[i] = l
    del(l)

for i in range(0,df.shape[0]):
    a = np.array(df.actors.iloc[i]).reshape(1,-1)
    l = []
    for j in range(0,df.shape[0]):
        b = np.array(df.actors.iloc[j]).reshape(1,-1)
        y = cosine_similarity(a,b)
        z = y[0][0]
        l.append(z)
    df['similarity_actors'].iloc[i] = l
    del(l)

for i in range(0,df.shape[0]):
    a = np.array(df.overview.iloc[i]).reshape(1,-1)
    l = []
    for j in range(0,df.shape[0]):
        b = np.array(df.overview.iloc[j]).reshape(1,-1)
        y = cosine_similarity(a,b)
        z = y[0][0]
        l.append(z)
    df['similarity_overview'].iloc[i] = l
    del(l)

df.head(10)

In [135]:
df1 = pd.read_csv('similarity-genres.csv')
df2 = pd.read_csv('similarity-actors.csv')
df3 = pd.read_csv('similarity-overview.csv')

In [136]:
df1.head(2)

,Unnamed: 0,id,title,genres,overview,vote,actors,director,similarity_genres
0,0,19995,Avatar,"[0.0, 0.05263157894736842, 0.10526315789473684...","[0.0, 2.9169826731229216e-05, 5.83396534624584...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000,"[0.9999999999999998, 0.2390457218668787, 0.583..."
1,1,285,Pirates of the Caribbean: At World's End,"[0.05263157894736842, 0.10526315789473684, 0.0...","[0.0007584154950119596, 0.0007875853217431889,...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426,"[0.2390457218668787, 0.9999999999999999, 0.216..."


In [137]:
df1.drop(columns=['Unnamed: 0'],inplace=True)

In [138]:
df2.head(2)

,Unnamed: 0,id,title,genres,overview,vote,actors,director,similarity_actors
0,0,19995,Avatar,"[0.0, 0.05263157894736842, 0.10526315789473684...","[0.0, 2.9169826731229216e-05, 5.83396534624584...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000,"[0.9999999999999999, 0.8854377448471462, 0.842..."
1,1,285,Pirates of the Caribbean: At World's End,"[0.05263157894736842, 0.10526315789473684, 0.0...","[0.0007584154950119596, 0.0007875853217431889,...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426,"[0.8854377448471462, 1.0, 0.9963692477452067, ..."


In [139]:
df2.drop(columns=['Unnamed: 0','title','genres','overview','vote','director','actors'],inplace=True)

In [140]:
df2.head(2)

,id,similarity_actors
0,19995,"[0.9999999999999999, 0.8854377448471462, 0.842..."
1,285,"[0.8854377448471462, 1.0, 0.9963692477452067, ..."


In [141]:
df3.head(2)

,Unnamed: 0,id,title,genres,overview,vote,actors,director,similarity_genres,similarity_actors,similarity_overview
0,0,19995,Avatar,"[0.0, 0.05263157894736842, 0.10526315789473684...","[0.0, 2.9169826731229216e-05, 5.83396534624584...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000,[],[],"[1.0, 0.67010169019074, 0.6297567769981597, 0...."
1,1,285,Pirates of the Caribbean: At World's End,"[0.05263157894736842, 0.10526315789473684, 0.0]","[0.0007584154950119596, 0.0007875853217431889,...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426,[],[],"[0.67010169019074, 1.0, 0.6914010178937784, 0...."


In [142]:
df3.drop(columns=['Unnamed: 0','title','genres','overview','vote','director','actors','similarity_genres','similarity_actors'],axis=1,inplace=True)

In [143]:
df3.head(2)

,id,similarity_overview
0,19995,"[1.0, 0.67010169019074, 0.6297567769981597, 0...."
1,285,"[0.67010169019074, 1.0, 0.6914010178937784, 0...."


In [144]:
df = df1.merge(df2,on='id')

In [145]:
df = df.merge(df3,on='id')

In [146]:
df.head(2)

,id,title,genres,overview,vote,actors,director,similarity_genres,similarity_actors,similarity_overview
0,19995,Avatar,"[0.0, 0.05263157894736842, 0.10526315789473684...","[0.0, 2.9169826731229216e-05, 5.83396534624584...",7.2,"[0.0, 0.00017599436818021823, 0.00035198873636...",0.000000,"[0.9999999999999998, 0.2390457218668787, 0.583...","[0.9999999999999999, 0.8854377448471462, 0.842...","[1.0, 0.67010169019074, 0.6297567769981597, 0...."
1,285,Pirates of the Caribbean: At World's End,"[0.05263157894736842, 0.10526315789473684, 0.0...","[0.0007584154950119596, 0.0007875853217431889,...",6.9,"[0.0005279831045406547, 0.0007039774727208729,...",0.000426,"[0.2390457218668787, 0.9999999999999999, 0.216...","[0.8854377448471462, 1.0, 0.9963692477452067, ...","[0.67010169019074, 1.0, 0.6914010178937784, 0...."


In [147]:
df.drop(columns=['genres','overview','actors'],axis=1,inplace=True)

In [148]:
df.head(2)

,id,title,vote,director,similarity_genres,similarity_actors,similarity_overview
0,19995,Avatar,7.2,0.000000,"[0.9999999999999998, 0.2390457218668787, 0.583...","[0.9999999999999999, 0.8854377448471462, 0.842...","[1.0, 0.67010169019074, 0.6297567769981597, 0...."
1,285,Pirates of the Caribbean: At World's End,6.9,0.000426,"[0.2390457218668787, 0.9999999999999999, 0.216...","[0.8854377448471462, 1.0, 0.9963692477452067, ...","[0.67010169019074, 1.0, 0.6914010178937784, 0...."


In [149]:




# def recommend_genres(movie):
#     ind = (df[df.title == movie].index)[0]
#     x = sorted(list(enumerate(ast.literal_eval(df[df.title == movie].similarity_genres[ind]))), reverse = True, key = lambda x:x[1])
#     y = x[1:6]
#     index = []
#     title = []
#     id = []
#     for i in range(0,5):
#         index.append(y[i][0])
#     for i in range(0,5):
#         id.append(df['id'].iloc[index[i]])
#     return id

# def recommend_actors(movie):
#     ind = (df[df.title == movie].index)[0]
#     x = sorted(list(enumerate(ast.literal_eval(df[df.title == movie].similarity_actors[ind]))), reverse = True, key = lambda x:x[1])
#     y = x[1:6]
#     index = []
#     title = []
#     id = []
#     for i in range(0,5):
#         index.append(y[i][0])
#     for i in range(0,5):
#         id.append(df['id'].iloc[index[i]])
#     return id

# def recommend_overview(movie):
#     ind = (df[df.title == movie].index)[0]
#     x = sorted(list(enumerate(ast.literal_eval(df[df.title == movie].similarity_overview[ind]))), reverse = True, key = lambda x:x[1])
#     y = x[1:6]
#     index = []
#     title = []
#     id = []
#     for i in range(0,5):
#         index.append(y[i][0])
#     for i in range(0,5):
#         id.append(df['id'].iloc[index[i]])
#     return id

In [155]:
# Recommendation function
def recommend_genres(movie):
    ind = (df[df.title == movie].index)[0]
    x = sorted(list(enumerate(ast.literal_eval(df[df.title == movie].similarity_genres[ind]))), reverse = True, key = lambda x:x[1])
    y = x[1:6]
    index = []
    title = []
    id = []
    poster_path = []
    for i in range(0,5):
        index.append(y[i][0])
    for i in range(0,5):
        id.append(df['id'].iloc[index[i]])
        
    return id


def recommend_actors(movie):
    ind = (df[df.title == movie].index)[0]
    x = sorted(list(enumerate(ast.literal_eval(df[df.title == movie].similarity_actors[ind]))), reverse = True, key = lambda x:x[1])
    y = x[1:6]
    index = []
    title = []
    id = []
    poster_path = []
    for i in range(0,5):
        index.append(y[i][0])
    for i in range(0,5):
        id.append(df['id'].iloc[index[i]])
        
    return id


def recommend_overview(movie):
    ind = (df[df.title == movie].index)[0]
    x = sorted(list(enumerate(ast.literal_eval(df[df.title == movie].similarity_overview[ind]))), reverse = True, key = lambda x:x[1])
    y = x[1:6]
    index = []
    title = []
    id = []
    poster_path = []
    for i in range(0,5):
        index.append(y[i][0])
    for i in range(0,5):
        id.append(df['id'].iloc[index[i]])
    return id


In [161]:
df.head(5)

,id,title,vote,director,similarity_genres,similarity_actors,similarity_overview,Recommended_genres,Recommended_actors,Recommended_overview
0,19995,Avatar,7.2,0.000000,"[0.9999999999999998, 0.2390457218668787, 0.583...","[0.9999999999999999, 0.8854377448471462, 0.842...","[1.0, 0.67010169019074, 0.6297567769981597, 0....","[49521, 127585, 1924, 8536, 27549]","[9366, 679, 10691, 10229, 9281]","[36668, 767, 9836, 10315, 297761]"
1,285,Pirates of the Caribbean: At World's End,6.9,0.000426,"[0.2390457218668787, 0.9999999999999999, 0.216...","[0.8854377448471462, 1.0, 0.9963692477452067, ...","[0.67010169019074, 1.0, 0.6914010178937784, 0....","[9713, 18947, 7350, 72358, 75531]","[58, 11109, 2575, 2977, 16300]","[297761, 2268, 254, 57158, 16558]"
2,206647,Spectre,6.3,0.000853,"[0.5833833511969478, 0.21693045781865616, 1.0,...","[0.8426548012644086, 0.9963692477452067, 1.000...","[0.6297567769981597, 0.6914010178937784, 1.000...","[1637, 59859, 10179, 43074, 290595]","[1598, 821, 11934, 9573, 16784]","[559, 71880, 8487, 1452, 3093]"
3,49026,The Dark Knight Rises,7.6,0.001279,"[0.9746318461970762, 0.40771775315010045, 0.66...","[0.823496020247966, 0.9928219197008146, 0.9993...","[0.45257961325551355, 0.5475657116810044, 0.54...","[9615, 136797, 13389, 8838, 2034]","[13909, 12163, 1931, 16784, 49529]","[40185, 9327, 209112, 2454, 91314]"
4,49529,John Carter,6.1,0.001705,"[0.5916079783099615, 0.282842712474619, 0.9970...","[0.8127181587124667, 0.9904058424754116, 0.998...","[0.45934444175602507, 0.50123915374135, 0.6089...","[99861, 68721, 54138, 188927, 14161]","[559, 558, 1931, 49026, 38757]","[1593, 1452, 559, 49538, 209112]"


In [158]:
df['Recommended_genres'] = [[]] * len(df)
df['Recommended_actors'] = [[]] * len(df)
df['Recommended_overview'] = [[]] * len(df)

In [159]:
for i in range(df.shape[0]):
    movie = df['title'].iloc[i]
    df['Recommended_genres'].iloc[i] = recommend_genres(movie)
    df['Recommended_actors'].iloc[i] = recommend_actors(movie)
    df['Recommended_overview'].iloc[i] = recommend_overview(movie)

C:\Users\Syed Raheel\AppData\Local\Temp\ipykernel_14152\1131928837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Recommended_genres'].iloc[i] = recommend_genres(movie)
C:\Users\Syed Raheel\AppData\Local\Temp\ipykernel_14152\1131928837.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Recommended_actors'].iloc[i] = recommend_actors(movie)
C:\Users\Syed Raheel\AppData\Local\Temp\ipykernel_14152\1131928837.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [ ]:
df.drop(columns=['director','similarity_genres','similarity_actors','similarity_overview','vote'],axis=1,inplace=True)

In [163]:
print(recommend_genres('Avatar'))
print(recommend_actors('Spider-Man 3'))
print(recommend_overview('Spider-Man 3'))

[49521, 127585, 1924, 8536, 27549]
[558, 38757, 49529, 3089, 99861]
[206647, 1452, 18360, 271110, 20662]


In [126]:
pickle.dump(df , open('Recommended.pkl' , 'wb'))